In [3]:
!poetry install --with vrp

Installing dependencies from lock file

Package operations: 3 installs, 0 updates, 0 removals

  - Installing texttable (1.7.0): Pending...
  - Installing texttable (1.7.0): Installing...
  - Installing texttable (1.7.0)
  - Installing igraph (0.11.9): Pending...
  - Installing igraph (0.11.9): Installing...
  - Installing igraph (0.11.9)
  - Installing leidenalg (0.10.2): Pending...
  - Installing leidenalg (0.10.2): Installing...
  - Installing leidenalg (0.10.2)

Installing the current project: ride_pfa (0.1.2)Installing the current project: ride_pfa (0.1.2)


In [ ]:
import sys
sys.path.append('../')

In [8]:
import numpy as np

from ride_vrp.configs import ModelConfig
from ride_vrp.data_model import *
from ride_vrp.initial_solution.pdptw_solution_builder import PDPTWSolutionBuilder
from ride_vrp.managers.pdptw_routing_manager_builder import PDPLiLimRoutingManagerBuilder
from ride_vrp.or_tools.callback import LoggingCallback
from ride_vrp.or_tools.model import VRPModelFactory
from ride_vrp.or_tools.model import VRPSolver

# Data parser

In [9]:
def parse_data(path: str) -> tuple[Node, list[Cargo], Tariff]:
    id2info = {}
    cargos = []

    with open(path, 'r') as file:
        for i, line in enumerate(file):
            line = line.split('\t')
            if i == 0:
                tariff = Tariff(
                    id='car',
                    capacity=np.array([int(line[1])]),
                    max_count=100,
                    cost_per_distance=[TariffCost(
                        min_dst_km=0,
                        max_dst_km=10000,
                        cost_per_km=1,
                        fixed_cost=0
                    )]
                )
            else:
                c_id = int(line[0])
                x = int(line[1])
                y = int(line[2])

                mass = int(line[3])

                et = int(line[4])
                lt = int(line[5])
                st = int(line[6])

                pick_up = int(line[7])
                delivery = int(line[8])
                if pick_up == delivery:
                    # print(12)
                    depo = Node(
                        id=0,
                        cargo_id=c_id,
                        demand=np.zeros([]),
                        service_time=0,
                        start_time=0,
                        end_time=lt,
                        coordinates=(x, y)
                    )
                    continue
                if pick_up == 0:
                    if c_id not in id2info:
                        id2info[c_id] = {}
                    id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
                else:
                    delivery = c_id
                    c_id = pick_up
                    if c_id not in id2info:
                        id2info[c_id] = {}
                    id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)

    for k, v in id2info.items():
        cargos.append(
            Cargo(
                id=k,
                nodes=[
                    Node(
                        cargo_id=k,
                        id=v[i][6] if i == 0 else v[i][7],
                        demand=np.array([v[i][2]]),
                        service_time=v[i][5],
                        start_time=v[i][3],
                        end_time=v[i][4],
                        coordinates=(v[i][0], v[i][1])
                    )
                    for i in range(2)
                ]
            )
        )

    return depo, cargos, tariff

In [10]:
depo, cargos, tariff = parse_data(f'../data/Li & Lim benchmark/pdp_100/lc101.txt')

# Calculate all distances

In [11]:
p2coordinates = {}
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates

distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [12]:
routing_manager = PDPLiLimRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
).with_tariffs(tariff).with_cargos(cargos).with_depo(depo).build()

In [13]:
model_factory = VRPModelFactory.get_pdptw_model_factory(routing_manager)

In [14]:
callback = LoggingCallback()

In [15]:
initial_solution_builder = PDPTWSolutionBuilder(
    solver=VRPSolver(
        model_config=ModelConfig(max_execution_time_minutes=1, max_solution_number=50),
        callback=callback
    ),
    model_factory=model_factory
)

In [16]:
solver = VRPSolver(
    model_config=ModelConfig(max_execution_time_minutes=1),
    initial_solution_builder=initial_solution_builder,
    callback=callback
)

In [17]:
import logging
import sys
root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s [%(levelname)s] - %(message)s')
handler.setFormatter(formatter)


In [18]:
sol = solver.solve(
    model_factory.build_model(routing_manager)
)

In [22]:
len([s for s in sol if len(s)])

10